In [2]:
import numpy as np
import scipy as sp
import scipy.linalg
import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import r2_score
import pickle
import time
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize
from scipy import sparse
import os.path
from scipy.interpolate import splrep, splev
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
sys.path.append('../..')
from CIMatrixLib.src.util import *
from CIMatrixLib.src.TreatPattern import *
from CIMatrixLib.src.algorithms.DebiasConvex import *
from CIMatrixLib.src.algorithms.CovariancePCA import *
from CIMatrixLib.src.algorithms.MCNNM import *
from CIMatrixLib.src.algorithms.RobustSyntheticControl import *
import CIMatrixLib.src.readData as readData
from CIMatrixLib.src.algorithms.OLS import *
from CIMatrixLib.src.algorithms.SDID import *
import importlib

sns.set_style("whitegrid")

In [3]:
import datetime as dt

In [4]:
train = pd.read_csv('dataset/train.csv')
store = pd.read_csv('dataset/store.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
display(train)
display(store)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [6]:
df = train
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['MonthName'] = df['Date'].dt.strftime('%b')
df['Year'] = df['Date'].dt.year
df['WeekofYear'] = df['Date'].dt.isocalendar().week
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Month,MonthName,Year,WeekofYear
0,1,5,2015-07-31,5263,555,1,1,0,1,7,Jul,2015,31
1,2,5,2015-07-31,6064,625,1,1,0,1,7,Jul,2015,31
2,3,5,2015-07-31,8314,821,1,1,0,1,7,Jul,2015,31
3,4,5,2015-07-31,13995,1498,1,1,0,1,7,Jul,2015,31
4,5,5,2015-07-31,4822,559,1,1,0,1,7,Jul,2015,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,1,Jan,2013,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1,1,Jan,2013,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1,1,Jan,2013,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1,1,Jan,2013,1


In [7]:
store = store[['Store', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']]
df = df.merge(store, on = ['Store'], how = 'inner')
display(df)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Month,MonthName,Year,WeekofYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec"


In [8]:
df['Month_Promo'] = df[['MonthName','PromoInterval']].apply(lambda x: x[0] in (str(x[1]).split(',')))

In [9]:
df['Month_Promo'] = df[['MonthName','PromoInterval']].apply(lambda x: x[0] in (str(x[1]).split(',')), axis = 1)
display(df)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Month,MonthName,Year,WeekofYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month_Promo
0,1,5,2015-07-31,5263,555,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN,False
1,1,4,2015-07-30,5020,546,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN,False
2,1,3,2015-07-29,4782,523,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN,False
3,1,2,2015-07-28,5011,560,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN,False
4,1,1,2015-07-27,6102,612,1,1,0,1,7,Jul,2015,31,0,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec",False
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec",False
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec",False
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,1,Jan,2013,1,1,22.0,2012.0,"Mar,Jun,Sept,Dec",False


In [10]:
df['Promo_new'] = df.Promo2.where(df.Month_Promo & ((df.Year-df.Promo2SinceYear)*100+(df.WeekofYear-df.Promo2SinceWeek)>=0), 0)

In [11]:
df['Promo_new'].sum()

146837

In [13]:
sale_matrix = pd.DataFrame(df.pivot_table(index = 'Store', columns = 'Date', values='Sales'))
display(sale_matrix)

Z = pd.DataFrame(df.pivot_table(index = 'Store', columns = 'Date', values='Promo_new'))
display(Z)

Z2 = pd.DataFrame(df.pivot_table(index = 'Store', columns = 'Date', values='Promo'))

import pickle 
pickle.dump('drug_sale.p', sale_matrix)

Date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,2013-01-10,...,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31
Store,,,,,,,,,,,,,,,,,,,,,
1,0.0,5530.0,4327.0,4486.0,4997.0,0.0,7176.0,5580.0,5471.0,4892.0,...,3464.0,3769.0,3706.0,4364.0,0.0,6102.0,5011.0,4782.0,5020.0,5263.0
2,0.0,4422.0,4159.0,4484.0,2342.0,0.0,6775.0,6318.0,6763.0,5618.0,...,5093.0,4108.0,3854.0,2512.0,0.0,6627.0,5671.0,6402.0,5567.0,6064.0
3,0.0,6823.0,5902.0,6069.0,4523.0,0.0,12247.0,9800.0,8001.0,7772.0,...,5414.0,5702.0,5080.0,3878.0,0.0,8107.0,8864.0,7610.0,8977.0,8314.0
4,0.0,9941.0,8247.0,8290.0,10338.0,0.0,12112.0,10031.0,8857.0,9472.0,...,8503.0,7286.0,8322.0,9322.0,0.0,11812.0,10275.0,10514.0,10387.0,13995.0
5,0.0,4253.0,3465.0,4456.0,1590.0,0.0,6978.0,5718.0,5974.0,4999.0,...,3595.0,3713.0,3815.0,2030.0,0.0,7059.0,6083.0,5899.0,4943.0,4822.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,0.0,5097.0,4579.0,4640.0,3325.0,0.0,9444.0,6472.0,5307.0,5887.0,...,4021.0,3587.0,3918.0,2177.0,0.0,7742.0,6793.0,4907.0,5263.0,5723.0
1112,0.0,10797.0,8716.0,9788.0,9513.0,0.0,25165.0,17058.0,14724.0,14366.0,...,6029.0,6730.0,6220.0,6216.0,0.0,14383.0,9583.0,9179.0,9652.0,9626.0
1113,0.0,6218.0,5563.0,5524.0,5194.0,0.0,8984.0,6866.0,6115.0,7508.0,...,4565.0,6410.0,6399.0,4784.0,0.0,7582.0,6468.0,6640.0,7491.0,7289.0


Date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,2013-01-10,...,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31
Store,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


TypeError: file must have a 'write' attribute

In [175]:
sale_matrix = sale_matrix.fillna(0).to_numpy()
remaining_cols = np.sum(sale_matrix==0, axis = 0) < 2000
remaining_rows = (np.sum(sale_matrix==0, axis = 1) < 2000) & (np.sum((Z.fillna(0)>0.5).to_numpy(), axis=1) != 0)

sale_matrix = sale_matrix[:, remaining_cols]
sale_matrix = sale_matrix[remaining_rows, :]

Z = (Z.fillna(0)>0.5).to_numpy()
Z = Z[:, remaining_cols]
Z = Z[remaining_rows, :]

Z2 = (Z2.fillna(0)>0.5).to_numpy()
Z2 = Z2[:, remaining_cols]
Z2 = Z2[remaining_rows, :]

In [176]:
Z.shape

(571, 942)

In [177]:
Z3 = (Z2 + Z >= 1)

In [1]:
#plt.imshow(sale_matrix == 0, cmap = 'Greys')
#plt.savefig('sales.eps', format='eps', dpi = 300)
#plt.show()

In [2]:
#plt.imshow(sale_matrix)
#sns.heatmap(sale_matrix)
#plt.savefig('sales.eps', format='eps', dpi = 300)
#plt.show()

In [21]:
np.mean(sale_matrix, axis = 1)

array([ 4755.90629012,  4945.59435173,  6942.56867779,  9638.33119384,
        4645.37355584,  5480.15917843,  8832.91784339,  5536.00898588,
        6562.33761232,  5569.80231065,  8031.50962773,  7589.51476252,
        5508.56739409,  6669.88960205,  7660.96405648,  6315.68934531,
        6541.61103979,  6464.91527599,  5468.24518614,  5520.28112965,
        9388.28626444,  6645.08344031,  9478.57381258,  5203.4762516 ,
        7364.77535302,  5281.18870347,  5862.6957638 ,  8436.65982028,
        8030.03851091,  9703.07188703,  7065.54557125,  6004.64313222,
        4852.57124519,  4883.45956354, 10178.13607189,  6822.9139923 ,
        5479.68164313,  5348.46983312,  7146.09627728,  3934.67779204,
        7285.36071887,  4256.07060334,  5265.96020539,  7882.48267009,
        4572.13735558,  7319.96148909, 10390.07060334,  5485.54685494,
        7987.95121951,  4709.88189987,  6503.86649551,  6955.28241335,
       10359.52503209,  5083.28112965,  6098.66238768,  7808.83055199,
      

In [3]:
#plt.imshow(Z, cmap = 'Greys', interpolation='nearest')
#plt.axis('off')
#plt.savefig('rossmann_pattern.png', dpi = 300)
#plt.show()

In [38]:
np.sum(Z, axis = 0)

array([162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 162,
       162, 162, 162, 162, 162, 162, 162, 162, 162, 164, 164, 164, 164,
        77,  77,  77,  77,  77,  77,  77,  77,  77,  77,  77,  77,  77,
        77,  77,  77,  77,  77,  77,  77,  77,  77,  77,  77,  56,  56,
        58,  58,  58,  58,  58,  58,  58,  58,  58,  58,  58,  58,  58,
        58,  58,  58,  58,  58,  58,  58,  58,  58,  58, 170, 170, 170,
       170, 170, 170, 170, 170, 170, 170, 170, 170, 170, 170, 170, 170,
       170, 170, 170, 170, 170, 170, 170, 170, 170,  80,  80,  80,  80,
        80,  80,  80,  80,  80,  80,  80,  80,  80,  80,  80,  80,  80,
        80,  80,  80,  80,  80,  80,  60,  60,  60,  60,  60,  60,  60,
        60,  60,  60,  60,  60,  60,  60,  60,  60,  60,  60,  60,  60,
        60,  60,  60,  60,  60, 172, 172, 172, 172, 172, 172, 172, 172,
       172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172, 172,
       172, 172, 172, 185, 185, 185,  98,  98,  98,  98,  98,  9

In [39]:
np.sum(sale_matrix==0) / sale_matrix.size

0.002478863264131734

In [42]:
np.sum(Z==1) / Z.size

0.29787462769623807

In [25]:
O = np.maximum(np.log(sale_matrix+1e-7), 0)

In [26]:
np.sum(O != 0) / O.size

0.9979090873418502

In [393]:
np.random.seed(3)
Omega = (1 - (np.random.rand(O.shape[0], O.shape[1]) < 0.2) * Z)*(O!=0)

In [92]:
%run ./../src/algorithms/DebiasConvexMissing.py
%run ./../src/algorithms/OLS.py

In [65]:
de_mean_O = True

old_O = O
if (de_mean_O):
    O = ((O - np.mean(O, axis=0)/2).T - np.mean(O, axis=1)/2).T
    #O = (O.T - np.mean(O, axis = 1)).T
    bias_O = old_O - O
else:
    bias_O = 0


In [380]:
print(O)

[[6.50428817 6.35957387 6.42810527 ... 6.25958146 6.30261898 6.31896811]
 [6.47697236 6.31896811 6.3526294  ... 6.58892648 6.39859493 6.43775165]
 [6.69084228 6.58063914 6.5366916  ... 6.57368017 6.7129562  6.71052311]
 ...
 [6.62406523 6.57646957 6.4707995  ... 6.37842618 6.57368017 6.57925121]
 [8.13182479 8.07433769 8.01829614 ... 8.17329344 8.08671792 8.2281769 ]
 [5.72031178 5.70378247 5.78689738 ... 6.15909539 6.21860012 6.28785856]]


In [273]:
print(np.sum(Omega)/Omega.size)

0.9131766737111177


In [178]:
O = sale_matrix

print(np.sum(O * (1-Z) * (O != 0)) / np.sum((1-Z) * (O != 0)))

print(np.sum(O * Z * (O != 0)) / np.sum(Z * (O != 0)))

6583.831003807696
6498.052274125415


In [152]:
def R2(A, B):
    A = A[A != 0]
    B = B[B != 0]
    return 1 - np.sum((A-B)**2) / np.sum((B-np.mean(B))**2)

In [170]:
M = O
tau = 50
mean_O_square = np.mean(O)**2
correct_ratio = np.sum(O*Z*(O!=0)) / np.sum(Z*(O!=0)) / np.sum(O*(1-Z)*(O!=0)) * np.sum((1-Z)*(O!=0))

In [154]:
mean_O = np.sum(O) / np.sum((O!=0)) 

In [155]:
correct_ratio

0.9869713044528828

In [184]:
np.random.seed(1)

results = {}
results['valid_error'] = []
#results['bias'] = []
results['tau'] = []
results['sum_of_square_Z'] = []
results['sum_of_square_1-Z'] = []
#results['mean_absolute'] = []
#results['R2'] = []

    
for t in range(10):
    Omega = (1 - (np.random.rand(O.shape[0], O.shape[1]) < 0.2)*Z) * (O!=0)
   
    #l = 4.2 
    l = 30000
    n = O.shape[0]
    m = O.shape[1]
    while (True):
        M, tau, info = convex_algorithm_with_Omega(O, Omega, Z*Omega, l, suggest = [M, tau], eps = 1e-1, debug = True)
        tau_debias, M_debias = debias(M, tau, Z*Omega, l)
        print('l=', l)
        print(tau_debias, tau)
        r = np.linalg.matrix_rank(M)
        #print(np.log(np.sum(((1-Z)*(M_debias-O))**2) / np.sum(1-Z)))
        #print( r * ((n+m) / (n*m)) * np.log((n*m) / (n+m)))
        #print(np.log(np.sum(((1-Z)*(M_debias-O))**2) / np.sum(1-Z)) + r * ((n+m) / (n*m)) * np.log((n*m) / (n+m)))
        #print(np.linalg.norm((O!=0)*(Omega)*(M_debias+tau_debias*Z-O)) / np.linalg.norm((O!=0)*(Omega)*O))

        print(np.linalg.norm((O!=0)*(1-Omega)*(M_debias+tau_debias*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
        print(np.sum((O!=0)*(1-Omega)*(1-Z)*(M_debias-O)) / np.sum((O!=0)*(1-Omega)*(1-Z)))
        print(np.sum((O!=0)*(1-Omega)*(Z)*(M_debias+Z*tau_debias-O)) / np.sum((O!=0)*(1-Omega)*(Z)))

        #print(np.sum(((O!=0)*(1-Omega)*(M_debias+tau_debias*Z-O))**2) / np.sum((O!=0)*(1-Omega)))

        #print(np.sum(np.abs((O!=0)*(1-Omega)*(M_debias+tau_debias*Z-O))) / np.sum((O!=0)*(1-Omega)))

        #print(R2((O!=0)*(1-Omega)*(M_debias+tau_debias*Z), (O!=0)*(1-Omega)*O))

        #print(np.sum((1-Omega)*(M_debias)) / np.sum(1-Omega))

        print('rank=', r)
        if (r > 70):
            break
        print()
        l = l / 1.05

    results['valid_error'].append(np.linalg.norm((O!=0)*(1-Omega)*(M_debias+tau_debias*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
    #results['bias'].append(np.sum((O!=0)*(1-Omega)*(Z)*(M_debias+Z*tau_debias-O)) / np.sum((O!=0)*(1-Omega)*(Z)))
    results['tau'].append(tau_debias)
    results['sum_of_square_Z'].append(np.sum(((O!=0)*(1-Omega)*Z*(M_debias+tau_debias*Z-O))**2) / np.sum((O!=0)*Z*(1-Omega)*((O-mean_O)**2)))
    results['sum_of_square_1-Z'].append(np.sum(((O!=0)*(1-Omega)*(1-Z)*(M_debias+tau_debias*Z-O))**2) / np.sum((O!=0)*(1-Z)*(1-Omega)) )
    #results['mean_absolute'].append(np.sum(np.abs((O!=0)*(1-Omega)*(M_debias+tau_debias*Z-O))) / np.sum((O!=0)*(1-Omega)))
    # #results['R2'].append(R2((O!=0)*(1-Omega)*(M_debias+tau_debias*Z), (O!=0)*(1-Omega)*O))

147.60327024962777
148.05742316228134
148.38836094967925
148.63885862684165
148.83500943168173
148.99235987033748
149.12099485388612
149.22789898961514
l= 30000
112.16734710374052 149.22789898961514
0.07831980794418862
nan
-3.6529175068033246
rank= 48

148.92547550050347
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars
148.71066878545787
148.5534810010919
148.43594801887238
l= 28571.42857142857
114.15349932931527 148.43594801887238
0.07804870423428911
nan
-3.1664269555425664
rank= 51

147.97407274648373
147.65355214364027
147.41013350598618
147.22121270747033
147.0739887583408
146.95957337606484
l= 27210.884353741494
115.74898854564051 146.95957337606484
0.0778154688209398
nan
-3.3364645780484947
rank= 55

146.53086918883926
146.23198155209258
146.0033345856733
145.8242465732213
145.68315686349902
145.5721055249894
l= 25915.127955944277
116.72138586969842 145.5721055249894
0.0775633084607067
nan
-3.02161131

In [185]:
pd.DataFrame(results).describe()

,valid_error,tau,sum_of_square_Z,sum_of_square_1-Z
count,10.000000,10.000000,10.000000,0.0
mean,0.077471,118.223946,0.043404,NaN
std,0.002234,2.384238,0.002362,NaN
min,0.074993,113.128124,0.040625,NaN
25%,0.075942,117.165107,0.041689,NaN
50%,0.077002,118.238994,0.043000,NaN
75%,0.078141,119.430395,0.044202,NaN
max,0.081965,121.560479,0.048125,NaN


In [339]:
print(np.linalg.matrix_rank(M))

18


In [162]:

M = np.zeros_like(O)
a = np.zeros((M.shape[0], 1))
b = np.zeros((M.shape[1], 1))

In [186]:
#l = 5
np.random.seed(1)
tau = 50

results = {}
results['valid_error'] = []
#results['bias'] = []
results['tau'] = []
results['sum_of_square_Z'] = []
results['sum_of_square_1-Z'] = []
#results['mean_absolute'] = []
#results['R2'] = []


for t in range(10):
    Omega = (1 - (np.random.rand(O.shape[0], O.shape[1]) < 0.3)*Z) * (O!=0)
    l = 28000
    while (True):
        M, a, b, tau = missing_algorithm(O, Omega*(1-Z), l = l, fixed_effects=True, suggest = [M, a, b])
        print('rank = ', np.linalg.matrix_rank(M))
        one_row = np.ones((1, M.shape[1]))
        one_col = np.ones((M.shape[0], 1))
        M1 = M + a.dot(one_row) + one_col.dot(b.T)
        tau=np.sum(Omega*Z*(O-M1))/np.sum(Omega*Z)
        print('tau=', tau)
        print(np.linalg.norm((O!=0)*(1-Omega)*(M1+tau*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
        print(np.sum(np.abs((O!=0)*(1-Omega)*(M1+tau*Z-O)/(O+1e-5))) / np.sum((O!=0)*(1-Omega)))

        #print(np.linalg.norm((Z)*(M1+tau*Z-O)) / np.linalg.norm((Z)*O))
        print(np.sum((O!=0)*(1-Omega)*(Z)*(M1+Z*tau-O)) / np.sum((O!=0)*(1-Omega)*(Z)))
        print(np.sum((O!=0)*(1-Omega)*(1-Z)*(M1-O)) / np.sum((O!=0)*(1-Omega)*(1-Z)))

        print(np.sum(((O!=0)*(1-Omega)*(M1+tau*Z-O))**2) / np.sum((O!=0)*(1-Omega)))
        #print(R2((O!=0)*(1-Omega)*(M1+tau*Z), (O!=0)*(1-Omega)*O))

        print()
        l = l / 1.1

        r = np.linalg.matrix_rank(M)
        if (r > 70):
            break

    results['valid_error'].append(np.linalg.norm((O!=0)*(1-Omega)*(M1+tau*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
    #results['bias'].append(np.sum((O!=0)*(1-Omega)*(Z)*(M1+Z*tau-O)) / np.sum((O!=0)*(1-Omega)*(Z)))
    #results['bias'].append(np.sum((O!=0)*(1-Omega)*(1-Z)*(M1-O)) / np.sum((O!=0)*(1-Omega)*(1-Z)))
    results['tau'].append(tau)
    results['sum_of_square_Z'].append(np.sum(((O!=0)*(1-Omega)*Z*(M1+tau*Z-O))**2) /  np.sum((O!=0)*Z*(1-Omega)*((O-mean_O)**2)))
    results['sum_of_square_1-Z'].append(np.sum(((O!=0)*(1-Omega)*(1-Z)*(M1+tau*Z-O))**2) / np.sum((O!=0)*(1-Z)*(1-Omega)))
    #results['R2'].append(R2((O!=0)*(1-Omega)*(M1+tau*Z), (O!=0)*(1-Omega)*O))


rank =  39
tau= -49.344501080904955
0.10163563029721315
0.07927069748682591
3.2271527022235915
nan
502243.9286677714

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
rank =  47
tau= -49.39798675385443
0.10077312984144667
0.07844285629011481
3.1455839363705977
nan
493755.8115406329

rank =  55
tau= -49.542224240737646
0.10004126511959185
0.07774096424023415
3.037007971739661
nan
486610.0522881069

rank =  65
tau= -49.76177074994283
0.09935293637147453
0.07708160819280171
2.936922782801386
nan
479936.89811326744

rank =  75
tau= -50.024771565862636
0.0987746195334304
0.07652484240121549
2.8189849360690995
nan
474365.8944341515

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in double_scalars
rank =  39
tau= -47.091048818348234
0.10283516734672161
0.0795326389498797
6.342667565435502
nan
518523.4996934537

rank =  48
tau= -47.48662256196609
0.102

In [187]:
pd.DataFrame(results).describe()

,valid_error,tau,sum_of_square_Z,sum_of_square_1-Z
count,10.000000,10.000000,10.000000,0.0
mean,0.098971,-49.446850,0.070908,NaN
std,0.001475,0.976033,0.002192,NaN
min,0.097040,-51.066892,0.068179,NaN
25%,0.097830,-50.059587,0.069186,NaN
50%,0.098716,-49.337982,0.070650,NaN
75%,0.099977,-48.736546,0.072480,NaN
max,0.101644,-47.962036,0.074691,NaN


In [507]:
print(np.sum(Z)/Z.size)

0.5213127343868245


In [98]:
%run ./../src/algorithms/OLS.py

In [181]:
np.random.seed(1)
tau = 50

results = {}
results['valid_error'] = []
results['bias'] = []
results['tau'] = []
results['sum_of_square_Z'] = []

for t in range(10):
    Omega = (1 - (np.random.rand(O.shape[0], O.shape[1]) < 0.2)*Z) * (O!=0)
    M, tau = OLS_with_Omega(O, Omega, Z*Omega, tau_star = -0.5, debug=True)
    print(np.linalg.norm((O!=0)*(1-Omega)*(M+tau*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
    print(np.sum((O!=0)*(1-Omega)*(M+tau*Z-O)) / np.sum((O!=0)*(1-Omega)))
    print(tau)

    results['valid_error'].append(np.linalg.norm((O!=0)*(1-Omega)*(M+tau*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
    results['bias'].append(np.sum((O!=0)*(1-Omega)*(M+tau*Z-O)) / np.sum((O!=0)*(1-Omega)))
    results['tau'].append(tau)
    results['sum_of_square_Z'].append(np.sum(((O!=0)*(1-Omega)*Z*(M+tau*Z-O))**2) /  np.sum((O!=0)*Z*(1-Omega)*((O-mean_O)**2)))



-34.888801987515464
-43.982459940664
-47.30453276967162
0.1601472598876502
4.098287400215411
-47.30453276967162
-34.60721824604399
-43.442244149902805
-46.68331254950651
0.16106486912991158
9.46266569895046
-46.68331254950651
-35.685437924215194
-44.932128825575546
-48.30670767960753
0.15804819456895877
-1.491848551754604
-48.30670767960753
-34.44023906817903
-43.36188021590015
-46.64143419772543
0.15629876532284204
6.095289413948144
-46.64143419772543
-35.87183500656273
-45.241858902938894
-48.67466626055175
0.1613134593444511
-6.855222137850248
-48.67466626055175
-35.99339131616471
-45.210516708712774
-48.580848077429884
0.1574370429828865
-6.030024897681761
-48.580848077429884
-37.260738341747796
-47.01297525995277
-50.591619009379116
0.16050411190791847
-14.202629075622344
-50.591619009379116
-35.44206914831314
-44.71284768417338
-48.10940689657048
0.15871042323864257
4.086383361006196
-48.10940689657048
-35.903363854078734
-45.571766188235166
-49.109944226011606
0.1582780307781813

In [182]:
pd.DataFrame(results).describe()

,valid_error,bias,tau,sum_of_square_Z
count,10.000000,10.000000,10.000000,10.000000
mean,0.159010,-0.321126,-48.090145,0.182758
std,0.001665,7.556368,1.249048,0.003397
min,0.156299,-14.202629,-50.591619,0.176465
25%,0.158106,-5.756661,-48.651212,0.181162
50%,0.158502,1.297267,-48.208057,0.182595
75%,0.160415,5.596039,-47.000365,0.185891
max,0.161313,9.462666,-46.641434,0.186874


In [105]:
M, tau = OLS(O, Z, tau_star = -0.5)
print(tau)

226.1662310343435


In [ ]:

M = np.zeros_like(O)
a = np.zeros((M.shape[0], 1))
b = np.zeros((M.shape[1], 1))

In [442]:
np.random.seed(1)
tau = 50

results = {}
results['valid_error'] = []
results['bias'] = []
results['tau'] = []
for t in range(10):
    Omega = (1 - (np.random.rand(O.shape[0], O.shape[1]) < 0.2)*Z) * (O!=0)
   
    #l = 4.2 
    l = 30000
    n = O.shape[0]
    m = O.shape[1]
    while (True):
        M, a, b, tau = convex_algorithm_with_Omega_with_fixed_effects(O, Omega, Z*Omega, l, suggest = [M, a, b, tau], debug = True)
        tau_debias, M_debias = debias(M, tau, Z*Omega, l)
        print('rank = ', np.linalg.matrix_rank(M))
        one_row = np.ones((1, M.shape[1]))
        one_col = np.ones((M.shape[0], 1))
        M1 = M_debias + a.dot(one_row) + one_col.dot(b.T)
        print('l=', l)
        print(tau_debias, tau)
        print(np.linalg.norm((O!=0)*(1-Omega)*(M1+tau_debias*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
        #print(np.linalg.norm((Z)*(M1+tau*Z-O)) / np.linalg.norm((Z)*O))
        print(np.sum((O!=0)*(1-Omega)*(Z)*(M1+Z*tau_debias-O)) / np.sum((O!=0)*(1-Omega)*(Z)))
        print(np.sum((O!=0)*(1-Omega)*(1-Z)*(M1-O)) / np.sum((O!=0)*(1-Omega)*(1-Z)))

        print()
        l = l / 1.1

        r = np.linalg.matrix_rank(M)
        #if (r > 90):
        #    break

    results['valid_error'].append(np.linalg.norm((O!=0)*(1-Omega)*(M1+tau_debias*Z-O)) / np.linalg.norm((O!=0)*(1-Omega)*O))
    #results['bias'].append(np.sum((O!=0)*(1-Omega)*(Z)*(M1+Z*tau-O)) / np.sum((O!=0)*(1-Omega)*(Z)))
    results['bias'].append(np.sum((O!=0)*(1-Omega)*(Z)*(M1+Z*tau_debias-O)) / np.sum((O!=0)*(1-Omega)*(Z)))
    results['tau'].append(tau_debias)

15.672033943871563
-1.8584659028071664
-13.345256520344536
rank =  64
l= 30000
8.266500151490986 -13.345256520344536
0.07411065249859686
12.46519787436826
nan

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars
-20.431560906391557
rank =  78
l= 27272.727272727272
-6.578052407273704 -20.431560906391557
0.07361430591559337
10.550891291941051
nan

-24.901907295265435
rank =  93
l= 24793.388429752064
-12.614296508711044 -24.901907295265435
0.07353042921469881
11.297342130118798
nan

-27.73901778706155
rank =  112
l= 22539.444027047328
-17.165757121239935 -27.73901778706155
0.0737267654883279
12.223226927472183
nan

-29.53154335191594
rank =  135
l= 20490.403660952114
-20.39097281461615 -29.53154335191594
0.07414690769209711
12.119784666093492
nan

-30.648836140990653
rank =  162
l= 18627.639691774646
-22.95674562712985 -30.648836140990653
0.07465705638179301
11.542184503967343
nan

-31.322221683956673
rank =  193

KeyboardInterrupt: 

In [440]:
pd.DataFrame(results).describe()

,valid_error,bias,tau
count,10.000000,10.000000,10.000000
mean,0.073356,3.180743,-34.455351
std,0.002191,5.869564,12.787340
min,0.070582,-6.256288,-41.083073
25%,0.071875,-0.685399,-40.116202
50%,0.072574,5.122857,-39.783527
75%,0.075221,6.760864,-37.666493
max,0.076784,12.522141,-0.107766


In [219]:
M_debias, tau_debias, info = non_convex_algorithm_with_Omega(O, Omega, Z, 400, tau = 44, debug = True)
print(np.linalg.norm((1-Omega)*(M_debias+tau_debias*Z-O)) / np.linalg.norm((1-Omega)*O))


50.14290596306389
53.59888075826649
54.25817381585397
52.857927929568596
50.247609866302966
47.04748195178352
43.632552483072125
40.212185492467654
36.897875156179325
33.74507494895717
30.777394988118118
28.00063065030144
25.410893238589427
22.999267077451215
20.754460254036392
18.664323165877125
16.716723267279622
14.900037687700035
13.203407995425842
11.616844776618384
10.131240608359114
8.738331632839069
7.430634216619525
6.201372776292886
5.044407581308919
3.9541668054681387
2.9255845721614127
1.9540454688840834
1.0353354126031005
0.16559847603654926
-0.6587008393267396
-1.440810449567207
-2.1837192294586427
-2.89018180628581
-3.562740130919064
-4.203742309050743
-4.815359131543152
-5.399598695948417
-5.958319462777881
-6.4932420380672635
-7.005959918762924
-7.497949382103181
-7.970578646420075
-8.425116383519445
-8.8627396245088
-9.284541074146558
-9.691535833706416
-10.084667528669549
-10.46481384220161
-10.832791466670491
-11.189360499209084
-11.535228321702691
-11.8710530177904

KeyboardInterrupt: 

In [341]:
print(tau_debias, tau)
print(np.linalg.norm((1-Omega)*(M_debias+tau_debias*Z-O)) / np.linalg.norm((1-Omega)*O))
#print(np.linalg.norm((1-Omega)*(M-O)) / np.linalg.norm((1-Omega)*O))

0.06592672264061933 0.10729437957564492
0.24319876814340388


In [ ]:
np.linalg.norm((M-O)*(1-Z)) / np.linalg.norm(M*(1-Z))

In [235]:
print(np.sum(O*Z3) / np.sum(Z3), np.sum(O*(1-Z3)) / np.sum(1-Z3))

6.575123082458817 6.005232359927543
